<a href="https://colab.research.google.com/github/pcsilcan/pcd/blob/master/20202/pcd_20202_0601_dining_philosophers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang-go

In [5]:
%%writefile dp.go
package main

import (
    "sync"
)

func philosopher(id int, rightFork, leftFork sync.Mutex) {
    for {
        // thinking
        rightFork.Lock()
        leftFork.Lock()
        // eat
        rightFork.Unlock()
        leftFork.Unlock()
    }
}

func main() {
    n := 5
    forks := make([]sync.Mutex, n)
    for i := 0; i < n -1 ; i++ {
        go philosopher(i, forks[i], forks[i + 1])
    }
    philosopher(n-1, forks[n-1], forks[0])
}

Overwriting dp.go


In [6]:
!go run dp.go

signal: interrupt


In [ ]:
!apt install spin

In [11]:
%%writefile dp1.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

byte forks[5] = { 1, 1, 1, 1, 1 }

active[5] proctype P() {
    do
    ::
        // think
        wait(forks[_pid])
        wait(forks[(_pid+1)%5])
        // eat
        signal(forks[_pid])
        signal(forks[(_pid+1)%5])
    od
}

Overwriting dp1.pml


In [13]:
!spin -run dp1.pml

pan:1: invalid end state (at depth 56)
pan: wrote dp1.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 52 byte, depth reached 57, errors: 1
       58 states, stored
       16 states, matched
       74 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.004	equivalent memory usage for states (stored*(State-vector + overhead))
    0.291	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds


In [14]:
!spin -t dp1.pml

spin: trail ends after 57 steps
#processes: 5
		forks[0] = 0
		forks[1] = 0
		forks[2] = 0
		forks[3] = 0
		forks[4] = 0
 57:	proc  4 (P:1) dp1.pml:11 (state 6)
 57:	proc  3 (P:1) dp1.pml:11 (state 6)
 57:	proc  2 (P:1) dp1.pml:11 (state 6)
 57:	proc  1 (P:1) dp1.pml:11 (state 6)
 57:	proc  0 (P:1) dp1.pml:11 (state 6)
5 processes created


In [15]:
%%writefile dp2.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

byte forks[5] = { 1, 1, 1, 1, 1 }
byte room = 4

active[5] proctype P() {
    do
    ::
        // think
        wait(room)
        wait(forks[_pid])
        wait(forks[(_pid+1)%5])
        // eat
        signal(forks[_pid])
        signal(forks[(_pid+1)%5])
        signal(room)
    od
}

Writing dp2.pml


In [16]:
!spin -run dp2.pml


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 52 byte, depth reached 2305, errors: 0
     3111 states, stored
     9280 states, matched
    12391 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.237	equivalent memory usage for states (stored*(State-vector + overhead))
    0.389	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.827	total actual memory usage


unreached in proctype P
	dp2.pml:19, state 16, "-end-"
	(1 of 16 states)

pan: elapsed time 0 seconds


In [17]:
%%writefile dp3.pml
#define wait(s) atomic { s > 0 -> s-- }
#define signal(s) s++

byte forks[5] = { 1, 1, 1, 1, 1 }

active[4] proctype P() {
    do
    ::
        // think
        wait(forks[_pid])
        wait(forks[(_pid+1)%5])
        // eat
        signal(forks[_pid])
        signal(forks[(_pid+1)%5])
    od
}

active proctype PL() {
    do
    ::
        // think
        wait(forks[0])
        wait(forks[4])
        // eat
        signal(forks[0])
        signal(forks[4])
    od
}

Writing dp3.pml


In [18]:
!spin -run dp3.pml


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 52 byte, depth reached 174, errors: 0
      243 states, stored
      568 states, matched
      811 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.019	equivalent memory usage for states (stored*(State-vector + overhead))
    0.290	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype P
	dp3.pml:16, state 12, "-end-"
	(1 of 12 states)
unreached in proctype PL
	dp3.pml:28, state 12, "-end-"
	(1 of 12 states)

pan: elapsed time 0 seconds
